This implementation is taken from https://medium.com/data-science/math-neural-network-from-scratch-in-python-d6da9f29ce65

In [161]:
import numpy as np
import os
import pandas as pd
import tensorflow

ModuleNotFoundError: No module named 'tensorflow'

In [123]:
#Base class
class Layer:
    def __init__self(self):
        self.input = None
        self.output = None

    def forward_propagation(self, input):
        raise NotImplementedError

    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [125]:
# Dense Layer class
class Dense(Layer):
    def __init__(self,input_size,output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward_propagation(self,input_data):
        self.input = input_data
        self.output = np.dot(self.input,self.weights) + self.bias
        return self.output

    def backward_propagation(self,output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        #dBias = output_error

        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [127]:
# Activation Layer class
class Activation(Layer):
    def __init__(self,activation,activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input)*output_error

In [129]:
# Some activation functions here
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

In [131]:
# Loss function MSE
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred,2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [222]:
# Network class
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self,layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []

        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result

    # train network
    def fit(self, x_train, y_train, epochs, learning_rate):
        samples= len(x_train)

        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss
                err += self.loss(y_train[j], output)
        
                # backward propagation
                error = self.loss_prime(y_train[j],output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            if i % 500 == 0:
                print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [147]:
# Test with XOR
# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(Dense(2, 3))
net.add(Activation(tanh, tanh_prime))
net.add(Dense(3, 1))
net.add(Activation(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=5000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/5000   error=0.783604
epoch 501/5000   error=0.000799
epoch 1001/5000   error=0.000265
epoch 1501/5000   error=0.000153
epoch 2001/5000   error=0.000106
epoch 2501/5000   error=0.000083
epoch 3001/5000   error=0.000066
epoch 3501/5000   error=0.000055
epoch 4001/5000   error=0.000047
epoch 4501/5000   error=0.000041
[array([[0.00031195]]), array([[0.99287769]]), array([[0.99025062]]), array([[0.0004473]])]


In [225]:
# Train with MNIST
# get data
dir = os.getcwd() + '\\data'
df = pd.read_csv(dir+'\\train.csv')

df = np.array(df).T
m,n = df.shape

yTrain = df[0]
yTrain = np.eye(10)[yTrain]

xTrain = df[1:n].T
xTrain = xTrain.reshape(xTrain.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255

In [227]:
# Network
net = Network()
net.add(Dense(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(Activation(tanh, tanh_prime))
net.add(Dense(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(Activation(tanh, tanh_prime))
net.add(Dense(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(Activation(tanh, tanh_prime))

net.use(mse, mse_prime)
net.fit(xTrain, yTrain, epochs=5000, learning_rate=0.1)

epoch 1/10   error=0.113038
epoch 2/10   error=0.095842
epoch 3/10   error=0.087705
epoch 4/10   error=0.084090
epoch 5/10   error=0.084980
epoch 6/10   error=0.080099
epoch 7/10   error=0.078800
epoch 8/10   error=0.079448
epoch 9/10   error=0.078286
epoch 10/10   error=0.074685
